<a href="https://colab.research.google.com/github/NermineGharbi24/Awesome-Spiking-Neural-Networks/blob/master/AnomalydetectionWindturbines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from getpass import getpass

# Ask for your token securely
token = getpass('Enter your GitHub token: ')


Enter your GitHub token: ··········


In [2]:
# ============================================================================
# COMPLETE GOOGLE COLAB SETUP - WIND FARM A DATASET
# Run cells in order: Cell 1 → Cell 2 → Cell 3 → Cell 4
# ============================================================================


# ============================================================================
# CELL 1: INSTALL REQUIRED LIBRARIES
# ============================================================================
!pip install kagglehub --quiet
!pip install snntorch --quiet
print("✓ Libraries installed")


# ============================================================================
# CELL 2: SETUP KAGGLE AND DOWNLOAD WIND FARM A
# ============================================================================
from google.colab import files
import os

print("=" * 70)
print("STEP 1: UPLOAD KAGGLE CREDENTIALS")
print("=" * 70)
print("Please upload your kaggle.json file...")
print("(Get it from: https://www.kaggle.com/settings → Create New API Token)")

uploaded = files.upload()

# Setup Kaggle credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("\n✓ Kaggle credentials configured")

print("\n" + "=" * 70)
print("STEP 2: DOWNLOADING DATASET")
print("=" * 70)
print("Dataset: Wind Turbine SCADA Data For Early Fault Detection")
print("Size: ~12.5 GB (Wind Farm A is part of this)")
print("⏰ Estimated time: 30-60 minutes")
print("⚠️  IMPORTANT: Keep this tab open and don't close your browser!")
print("=" * 70)

import kagglehub

# Download the entire dataset (includes Wind Farm A)
path = kagglehub.dataset_download("azizkasimov/wind-turbine-scada-data-for-early-fault-detection")

print(f"\n✓ Dataset downloaded to: {path}")

# Create easy access link
!ln -sf "{path}" ./data

print("\n" + "=" * 70)
print("VERIFYING DOWNLOAD")
print("=" * 70)

# Check if Wind Farm A exists
wind_farm_a_exists = os.path.exists(f"{path}/Wind Farm A")
print(f"Wind Farm A folder exists: {wind_farm_a_exists}")

if wind_farm_a_exists:
    print("\nWind Farm A contents:")
    !ls -lh "{path}/Wind Farm A/"

    print("\nTurbine files in Wind Farm A:")
    !ls -lh "{path}/Wind Farm A/datasets/" 2>/dev/null || echo "Checking alternative path..."

    # Count CSV files
    import glob
    csv_files = glob.glob(f"{path}/Wind Farm A/**/*.csv", recursive=True)
    print(f"\nTotal CSV files found: {len(csv_files)}")

    print("\nSample files:")
    for f in csv_files[:5]:
        print(f"  {f}")
else:
    print("❌ Wind Farm A folder not found!")
    print("Showing available folders:")
    !ls -lh "{path}/"

print("\n✓ Download verification complete")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 3.2 MB/s eta 0:00:00
✓ Libraries installed
STEP 1: UPLOAD KAGGLE CREDENTIALS
Please upload your kaggle.json file...
(Get it from: https://www.kaggle.com/settings → Create New API Token)


Saving kaggle.json to kaggle (1).json

✓ Kaggle credentials configured

STEP 2: DOWNLOADING DATASET
Dataset: Wind Turbine SCADA Data For Early Fault Detection
Size: ~12.5 GB (Wind Farm A is part of this)
⏰ Estimated time: 30-60 minutes
⚠️  IMPORTANT: Keep this tab open and don't close your browser!
Resuming download from 6829375488 bytes (5704913899 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/azizkasimov/wind-turbine-scada-data-for-early-fault-detection?dataset_version_number=2 (6829375488/12534289387) bytes left.


100%|██████████| 11.7G/11.7G [03:11<00:00, 29.8MB/s]

Extracting files...



✓ Dataset downloaded to: /root/.cache/kagglehub/datasets/azizkasimov/wind-turbine-scada-data-for-early-fault-detection/versions/2

VERIFYING DOWNLOAD
Wind Farm A folder exists: True

Wind Farm A contents:
total 24K
-rw-r--r-- 1 root root 1.7K Oct 14 12:37 comma_event_info.csv
-rw-r--r-- 1 root root 4.0K Oct 14 12:37 comma_feature_description.csv
drwxr-xr-x 2 root root 4.0K Oct 14 12:37 datasets
-rw-r--r-- 1 root root 1.8K Oct 14 12:37 event_info.csv
-rw-r--r-- 1 root root 4.1K Oct 14 12:37 feature_description.csv

Turbine files in Wind Farm A:
total 1.5G
-rw-r--r-- 1 root root 36M Oct 14 12:37 0.csv
-rw-r--r-- 1 root root 35M Oct 14 12:37 10.csv
-rw-r--r-- 1 root root 35M Oct 14 12:37 13.csv
-rw-r--r-- 1 root root 35M Oct 14 12:37 14.csv
-rw-r--r-- 1 root root 36M Oct 14 12:37 17.csv
-rw-r--r-- 1 root root 34M Oct 14 12:37 22.csv
-rw-r--r-- 1 root root 36M Oct 14 12:37 24.csv
-rw-r--r-- 1 root root 35M Oct 14 12:37 25.csv
-rw-r--r-- 1 root root 35M Oct 14 12:37 26.csv
-rw-r--r-- 1 roo

In [3]:
pip install snntorch

In [4]:
# ============================================================================
# CELL 3: IMPORT LIBRARIES FOR DATA PROCESSING
# ============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import snntorch as snn
from snntorch import surrogate

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (f1_score, precision_score, recall_score,
                             roc_auc_score, confusion_matrix, classification_report)
from tqdm import tqdm
import warnings
import random
import glob

warnings.filterwarnings('ignore')

# Set random seeds
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

print("=" * 70)
print("LIBRARIES IMPORTED")
print("=" * 70)
print(f"PyTorch version: {torch.__version__}")
print(f"snnTorch version: {snn.__version__}")
print(f"Device: {'GPU (CUDA)' if torch.cuda.is_available() else 'CPU'}")
print("=" * 70)

LIBRARIES IMPORTED
PyTorch version: 2.8.0+cu126
snnTorch version: 0.9.4
Device: CPU


In [ ]:
# ============================================================================
# CELL 4: LOAD ALL WIND FARM A DATA
# ============================================================================
print("\n" + "=" * 70)
print("LOADING WIND FARM A DATA")
print("=" * 70)

# Find Wind Farm A path
base_path = './data/Wind Farm A'

# Find all turbine CSV files
turbine_files = sorted(glob.glob(f'{base_path}/datasets/*.csv'))

if not turbine_files:
    # Try alternative paths
    turbine_files = sorted(glob.glob(f'{base_path}/**/*.csv', recursive=True))
    turbine_files = [f for f in turbine_files if '/datasets/' in f]

print(f"\nFound {len(turbine_files)} turbine files in Wind Farm A")

if len(turbine_files) == 0:
    print("❌ No turbine files found!")
    print("\nSearching in all locations:")
    all_csv = glob.glob('./data/**/*.csv', recursive=True)
    print(f"Total CSV files found: {len(all_csv)}")
    for f in all_csv[:10]:
        print(f"  {f}")
    print("\n⚠️  Please check the paths and update base_path variable")
else:
    print("\nTurbine files:")
    for f in turbine_files:
        size_mb = os.path.getsize(f) / (1024 * 1024)
        print(f"  {os.path.basename(f)}: {size_mb:.1f} MB")

    # Load all turbines
    print("\n" + "=" * 70)
    print("LOADING AND COMBINING TURBINES")
    print("=" * 70)

    signals_list = []

    for i, file in enumerate(turbine_files, 1):
        print(f"[{i}/{len(turbine_files)}] Loading {os.path.basename(file)}...", end=" ")

        try:
            df = pd.read_csv(file)
            turbine_id = os.path.basename(file).replace('.csv', '')
            df['turbine_id'] = turbine_id
            signals_list.append(df)
            print(f"✓ ({df.shape[0]:,} rows, {df.shape[1]} cols)")
        except Exception as e:
            print(f"❌ Error: {e}")

    if signals_list:
        # Combine all turbines
        print(f"\nCombining {len(signals_list)} turbines...")
        signals = pd.concat(signals_list, ignore_index=True)

        print("\n✓ ALL TURBINES LOADED!")
        print("=" * 70)
        print(f"Total samples: {len(signals):,}")
        print(f"Total columns: {len(signals.columns)}")
        print(f"Turbines: {sorted(signals['turbine_id'].unique())}")
        print(f"Memory usage: {signals.memory_usage(deep=True).sum() / (1024**2):.1f} MB")

        # Show sample
        print("\nFirst few rows:")
        print(signals.head())

        print("\nColumns:")
        print(signals.columns.tolist())

        # Identify time column
        print("\n" + "=" * 70)
        print("PROCESSING TIME COLUMN")
        print("=" * 70)

        time_candidates = [col for col in signals.columns
                          if any(x in col.lower() for x in ['time', 'date', 'stamp'])]

        if time_candidates:
            time_col = time_candidates[0]
            print(f"Time column found: {time_col}")
        else:
            time_col = signals.columns[0]
            print(f"Using first column as time: {time_col}")

        # Rename to standard name
        if time_col != 'time_stamp':
            signals.rename(columns={time_col: 'time_stamp'}, inplace=True)
            print(f"✓ Renamed to 'time_stamp'")

        # Convert to datetime
        signals['time_stamp'] = pd.to_datetime(signals['time_stamp'])

        # Sort by time
        signals = signals.sort_values('time_stamp').reset_index(drop=True)

        print(f"\nTime range:")
        print(f"  Start: {signals['time_stamp'].min()}")
        print(f"  End:   {signals['time_stamp'].max()}")
        print(f"  Duration: {signals['time_stamp'].max() - signals['time_stamp'].min()}")
        print(f"  Frequency: ~{len(signals) / ((signals['time_stamp'].max() - signals['time_stamp'].min()).total_seconds() / 60):.1f} samples/minute")

        # Load anomaly events
        print("\n" + "=" * 70)
        print("LOADING ANOMALY EVENTS")
        print("=" * 70)

        event_file = f'{base_path}/comma_event_info.csv'

        if os.path.exists(event_file):
            anomaly_events = pd.read_csv(event_file)
            print(f"✓ Loaded anomaly events from: {event_file}")
            print(f"\nShape: {anomaly_events.shape}")
            print("\nAnomaly events:")
            print(anomaly_events)

            # Convert to datetime
            if 'event_start' in anomaly_events.columns:
                anomaly_events['event_start'] = pd.to_datetime(anomaly_events['event_start'])
            if 'event_end' in anomaly_events.columns:
                anomaly_events['event_end'] = pd.to_datetime(anomaly_events['event_end'])
        else:
            print(f"⚠️  Event file not found: {event_file}")
            print("Creating synthetic anomaly events...")

            # Create 10 random anomaly periods
            total_samples = len(signals)
            anomaly_starts = np.random.choice(
                range(0, total_samples - 5000, 10000),
                size=10,
                replace=False
            )

            anomaly_events = pd.DataFrame({
                'event_start': [signals['time_stamp'].iloc[i] for i in anomaly_starts],
                'event_end': [signals['time_stamp'].iloc[i + np.random.randint(1000, 3000)]
                             for i in anomaly_starts]
            })

            print("\nCreated synthetic anomaly events:")
            print(anomaly_events)

        # Identify sensor columns
        print("\n" + "=" * 70)
        print("IDENTIFYING SENSOR COLUMNS")
        print("=" * 70)